In [0]:
!pip install mxnet-cu80 gluonnlp matplotlib tqdm 

## `NDArray`

NDArray는 동일한 데이터 타입 및 크기 (32 비트 부동 소수점, 32 비트 정수 등)를 가지는 n차원 배열이다. 이 배열이 중요한 이유는  딥러닝 학습 및 실행에 필요한 많은 수학 연산을 통한 데이터 저장을 할 수 있는 다차원 배열이기 때문이다. 입력 데이터, 가중치 및 출력 데이터는 벡터 및 행렬에 저장되므로 이러한 종류의 데이터 구조를 기반으로 하여야한다.



NDArray는 `numpy.ndarray`와 매우 유사하나 두 가지 큰 차이점이 존재한다.

1. 여러 디바이스에서 동작한다. : GPU, CPU 모두에서 같은 코드로 동작한다.
2. 모든 연산은 병렬로 수행된다.  : 자동으로 병렬화 연산이 된다.

#### NDArray 생성

Gluon과 MXNet 모두 NDArray(이하 nd)는 텐서(tensor)로 사용되고 이 텐서는 딥러닝 알고리즘에서 데이터를 내포하고 있다. 일반적으로 아래와 같은 형태로 Python 환경 상에서 활용된다.


In [0]:
import mxnet as mx
from mxnet import nd
import numpy as np

용어 정의를 해보자! nd에서 차원이라고 함은 축(axis)의 개수라고 정의한다. 따라서 아래는 2차원의 데이터(행렬)를가지는 nd객체이다.

In [0]:
a = nd.array([[1,2,3], [4,5,6]])
a

`nd.array`명령어는 주로 `list`형태의 객체를 받아들여 nd객체를 만들어준다. 이렇게 만들어진 nd객체를 출력하면 우리가 배웠던 행렬 형태로 출력해주고 다양한 정보를 알려준다.

`a`는 2차원 매트릭스이고 1차원에 2개, 2차원에 3개의 원소를 보유하고 있고, CPU 컨텍스트(context)에 존재한다는 사실을 알려주고 있다. 여기서 컨텍스트는 nd의 연산이 일어나는 영역이다.

> **컨텍스트(Context)**
> _nd객체는 대표적으로 CPU, GPU 컨텍스트를 가질 수 있고 머신에 다수의 GPU 카드가 존재하면 GPU 카드 개별의 컨텍스를 가진다.  서로 다른 컨텍스트를 가지고 있는 nd끼리의 연산은 불가능하며, 서로 다른 컨텍스트를 가질 경우  둘중 하나의 컨텍스트를 동일하게 맞춰주는 명령어로 연산 수행이 가능하다. 이렇게 제한을 가한 이유는 각 컨텍스트에서 충분한 병렬 연산으로 연산 최적화를 하기 위함이다._ 

In [0]:
a.shape, a.dtype, a.context

각 nd객체의 차원과 각 차원에 몇개의 원소를 보유하고 있는지 알수 있는 명령어로 `shape`명령어. 그리고 nd객체의 데이터 타입을 알수 있는 'dtype' 명령어, 총 원소 개수를 확인하는 `size`, 마지막으로 nd객체의 컨텍스트를 조회하는 `context`가 있다.

이런 특히 `shape`명령어의 결과는 많은 정보를 내포하고 있어 여러 함수의 인자로 사용되는데, 아래와 같은 nd객체 생성시에 주로 사용된다.

In [0]:
# 4 x 4의 0으로 채워진 매트릭스 생성
a = nd.zeros((4,4))
# 4 x 4의 1로 채워진 매트릭스 생성
b = nd.ones((4,4))
# 모든 값을 5로 변경
b[:] = 5
# b의 모든 값을 a로 복사한다.
b.copyto(a)
print(b)

다양한 데이터 타입 설정도 가능하다.

In [0]:
# 32비트 float형이 기본 데이터 타입으로 쓰임
a = nd.array([1,2,3])
# int32형
b = nd.array([1,2,3], dtype=np.int32)
# 16비트 float array
c = nd.array([1.2, 2.3], dtype=np.float16)
(a.dtype, b.dtype, c.dtype)

### Inspect an NDArray's Attributes

As with NumPy, the dimensions of each NDArray are accessible by accessing the `.shape` attribute. We can also query its `size`, which is equal to the product of the components of the shape. In addition, `.dtype` tells the data type of the stored values.

In [0]:
(a.shape, a.size, a.dtype, a.context)

모델링 실무시 많은경우  모든 원소를 직접 설정하기 보다 아래와 같이 차원을 명시해서 자동으로 생성해 사용한다. 

In [0]:
# 0으로 채워진 2차원 (2,3) 쉐입을 가지는 nd객체 생성  
a = nd.zeros((2,3))
# 같은 쉐입이나 1로 채워진 객체 생성
b = nd.ones((2,3))
# 7로 채워진 객체 생성
c = nd.full((2,3), 7)
# 객체 생성 영역의 메모리를 초기화 하지 않은 임의의 값을 가지는 객체 생성
d = nd.empty((2,3))

`asnumpy`명령어로 `numpy`객체로 변환.

In [0]:
b = nd.arange(18).reshape((3,2,3))
b.asnumpy()

#### NDArray 연산  

같은 쉐입을 가지는 nd객체들의 연산은 개별 원소들끼의 수치 연산이 수행된다.

이런 연산시 nd와 같은 도구 없이 수행해야 된다면 차원의 수에 따른 반복문이 중첩이 되어야 될 것이다. 코드가 늘어나는 이슈 뿐만 아니라 그 코드의 라인수만큼 늘어나는 프로그램 버그의 위험도 감수해야 된다.

In [0]:
#1로 채워지고 쉐입이 (4,3)인 행렬 생성
a = nd.ones((4,3))
b = nd.ones((4,3))
# 원소 합
c = a + b
# 원소 뺄셈
d = - c
# 원소 거듭제곱과 sine 함수 적용 그리고 행렬 전치
e = nd.sin(c**2).T
# 원소 최대값
f = nd.maximum(a, c)
f.asnumpy()

#### 인덱싱(Indexing)과 슬라이싱(slicing)

nd객체를 자르는 연산을 슬라이싱이라고 한다. 이러한 연산은 데이터를 부분적으로 획득하거나 덮어쓸때 매우 유용한 연산이다.  아마도 `numpy`에 익숙한 독자라면 큰 차이를 느끼지 못할 것이다.

In [0]:
a = nd.array([[1,2], [3,4]])
a[0:1]

아래와 같이 특정 값을 접근할 수 있다.

In [0]:
a[1,0]

#### 브로드캐스팅(Broadcasting)

브로드캐스팅이라는 말뜻은 뭔가를 전달한다는 의미이다. 이는 벡터나 행렬 연산시 하나의 벡터를 행렬의 행이나 열을 따라서 전달을 하면서 어떤 연산을 수행할 수 있다는 것을 의미한다.

In [0]:
x = nd.ones(shape=(3,2))
x

In [0]:
y = nd.arange(3).reshape((3,1))
x + y

위 코드는 x의 상위차원에 대해서 y의 벡터를 차례로 적용해서 계산한 결과를 보여준다. 브로드캐스팅은 작은 차원에 대해서 브로드캐스팅을 먼저 수행하려 하기 때문에 위와 같은 동작을 보여준다.

#### 벡터, 행렬 연산

##### 벡터 연산


벡터는`[0,1,2,3]` 과 같이  숫자의 리스트로 생각할 수 있다. nd객체로는 아래와 같이 표현된다.  

In [0]:
a = nd.arange(4)
a

In [0]:
a[1]

In [0]:
a.shape

In [0]:
a.sum()

In [0]:
a.mean()

0으로 시작되는 인덱스로 벡터 원소(스칼라값)에 접근할 수 있다. 그리고 합이나 평균과 같은 연산을 수행할 수 있다.

두 벡터의 내적은 $\sum_{i=1}^n a_i  \cdot b_i$와 같이 표현된다.


In [0]:
a = nd.arange(4)
b = nd.arange(4)
nd.dot(a,b)

두 벡터의 내적 연산은 일종의 유사성을 의미한다. 벡터의 크기가 1이라고 한다면 두 벡터의 내적은 두 벡터의 사이 각도의 코사인(cosine) 값인 코사인 유사도를 의미한다.

일반적으로 벡터 노름(norm)연산은 $\lVert A\rVert_2=\sqrt{x_1^2 + x_2^2 + \cdots + x_n^2}$와 같이 수행되며 이는 벡터의 크기를 의미하고 $l_2$norm 이라는 이름으로 불리운다. $l_p norm = \lVert A\rVert_p  = (\sum_{i=1}^d |x_i|^p)^{\frac{1}{p}}$와 같이 일반화 시킬 수 있다.

`a/a.norm()` 코드는 일종의 벡터 정규화를 의미하며 이렇게 변환된 벡터는 단위벡터(unit vector)라하며 벡터의 크기가 1이 된고, 결과적으로 두 단위벡터 연산시 방향성의 차이만가 계산되게 된다.


In [0]:
#자기 자신과의 코사인 유사도는 1에 가까운 값일 것이다.
a = nd.arange(4)
nd.dot(a/a.norm(), a/a.norm())

In [0]:
#벡터 원소의 합이 1 이라면 내적은 가중평균을 의미한다.
nd.dot(a/a.sum(), a)

딥러닝에서 이러한 내적(dot product) 연산은 유사도, 대표성의 의미 때문에 빈번히 수행되고 활용되니 이 연산이 어떠한 의미를 포함하는지 직관적으로 알아둘 필요가 있다.


행렬과 벡터의 내적도 아래와 같은 방식으로 동일하게 계산된다.

$$ A \cdot x=
    \begin{pmatrix}
    A_{11} & A_{12} & \cdots & A_{1m} \\
    A_{21} & A_{22} & \cdots & A_{2m} \\
    \vdots & \vdots & \ddots & \vdots  \\
    A_{n1} & A_{n2} & \cdots & A_{nm} \\
    \end{pmatrix} \cdot
    \begin{pmatrix}
    x_1 \\
    x_2 \\
    \vdots \\
    x_m
    \end{pmatrix} =
    \begin{pmatrix}
    A_{11} x_1 + \cdots  + A_{1m} x_m \\
    A_{21} x_1 + \cdots  + A_{2m} x_m \\
    \vdots  \\
    A_{n1} x_1 + \cdots  + A_{nm} x_m \\
    \end{pmatrix}
$$

In [0]:
a = nd.arange(4)
b = nd.ones((2,4))
nd.dot(b,a)  

##### 행렬연산

행렬을 열벡터만으로 표현할 수 있는데 그렇게 되면 아래와 같은 행렬을 다시 표현할 수 있다.

$$
A =
\begin{pmatrix}
A_{11} & A_{12} & \cdots & A_{1m} \\
A_{21} & A_{22} & \cdots & A_{2m} \\
\vdots & \vdots & \ddots & \vdots  \\
A_{n1} & A_{n2} & \cdots & A_{nm} \\
\end{pmatrix} =
\begin{pmatrix}
\mathbf {A_1}^T \\
\mathbf {A_2}^T \\
\vdots \\
\mathbf {A_n}^T \\
\end{pmatrix}
$$

$$
B =
\begin{pmatrix}
B_{11} & B_{12} & \cdots & B_{1k} \\
B_{21} & B_{22} & \cdots & B_{2k} \\
\vdots & \vdots & \ddots & \vdots  \\
B_{m1} & B_{n2} & \cdots & B_{mk} \\
\end{pmatrix} =
\begin{pmatrix}
\mathbf {B_1}^T  \mathbf {B_2}^T \cdots \mathbf {B_k}^T \\
\end{pmatrix}
$$

$$
C = AB =
\begin{pmatrix}
\mathbf {A_1}^T \\
\mathbf {A_2}^T \\
\vdots \\
\mathbf {A_n}^T \\
\end{pmatrix}
\begin{pmatrix}
\mathbf {B_1}^T  & \mathbf {B_2}^T & \cdots & \mathbf {B_k}^T \\
\end{pmatrix}$$ $$=
\begin{pmatrix}
\mathbf {A_1}^T  \mathbf {B_1}^T & \mathbf {A_1}^T  \mathbf {B_2}^T &  \cdots & \mathbf {A_1}^T  \mathbf {B_k}^T \\
\mathbf {A_2}^T  \mathbf {B_1}^T & \mathbf {A_2}^T  \mathbf {B_2}^T &  \cdots & \mathbf {A_2}^T  \mathbf {B_k}^T \\
\vdots & \vdots & \ddots &  \vdots \\
\mathbf {A_n}^T  \mathbf {B_1}^T & \mathbf {A_n}^T  \mathbf {B_2}^T &  \cdots & \mathbf {A_n}^T  \mathbf {B_k}^T \\
\end{pmatrix}
$$

위 수식과 같은 방식으로 계산이 수행되나 아래와 같은 간단한 함수로 계산이 된다. 게다가 행렬 연산은 개별 연산이 독립적으로 수행이 가능하기 때문에 병렬화에 용이한 측면이 있으며 nd에서는 이 계산 특징을 이용해 충분히 빠르게 연산을 수행해 준다.

위와 같은 행렬곱 연산을 위해서 `nd.dot` 함수를 이용하게 되며 일반적인 곱(`*`) 연산자를 사용하게 되면 원소별 곱 연산을 수행하게 된다.

In [0]:
a = nd.array([[1,2], [3,4]])
b = a
a * b

In [0]:
nd.dot(a,b)

대표적인 행렬 연산과 벡터 연산에 대해서 살펴봤다.  이러한 연산은 딥러닝 네트워크를 구성하는 아주 기본적연 연산이며, 더 상세한 내용은 별도의 선형대수에서 설명하고 있다. 지금까지 2차원의 행렬 연산만 소개했으나 실제 데이터는 3차원으로 주로 구성되고 많게는 5차원 행렬까지 보게 될 것이다. 간단하게 대표적인 데이터 형태 예시를 보도록 하자

- 벡터 : 2차원 nd. (데이터 샘플, 속성)
- 시계열 데이터 : 3차원 nd (데이터 샘플, 시간, 속성)
- 이미지 : 4차원 nd (데이터 샘플, 채널, 높이, 너비)
- 비디오 : 5차원 nd(데아터 샘플, 프레임, 채널, 높이, 너비)
